In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 12.9 MB/s 
     |████████████████████████████████| 182 kB 72.7 MB/s 
     |████████████████████████████████| 7.6 MB 49.9 MB/s 


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 11.4 MB/s 
     |████████████████████████████████| 115 kB 76.1 MB/s 
     |████████████████████████████████| 212 kB 76.4 MB/s 
     |████████████████████████████████| 127 kB 64.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [12]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 17.2 MB/s 


#1. 데이터 처리 작업

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "This course is amazing",
]
batch = tokenizer(sequences, padding = True, truncation = True, return_tensors = "pt")

batch['labels'] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## 허브에서 데이터셋 로딩

In [ ]:
from datasets import load_dataset

# 10가지 데이터셋으로 구성된 GLUE 벤치마크 중 하나: MRPC
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

train set은 3668개의 문장 쌍과 sentence1, sentence2, label, idx열로, 
validation set은 408개의 문장 쌍과 sentence1, sentence2, label, idx열로,
test set은 1725개의 문장 쌍과 sentence1, sentence2, label, idx 열로 구성되어있다.  

Python의 dictionary와 같이 key값을 이용하여 raw_datasets 개체의 개별 집합(train set, validation set, test set)에 접근할 수 있다.

In [ ]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

레이블은 이미 정수 인코딩이 되어있기 때문에 별도의 전처리가 필요하지 않다. 어떤 정수가 어떤 레이블에 해당하는지 파악하기 위해서는 raw_train_dataset의 features 속성을 살펴보면 된다. 

In [ ]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

레이블 1은 equivalent였다.

## 데이터셋 전처리


### 문장 토큰화

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets['train']['sentence1'])
tokenized_sentences_2 = tokenizer(raw_datasets['train']['sentence2'])

하지만 두 sequences를 모델에 바로 전달하여 두 문장이 의역인지 아닌지에 대한 예측은 불가하기 때문에 두 sequences를 pair로 처리하고 적절한 전처리를 적용해야 함. 다행히도 tokenizer는 다음과 같이 한 쌍의 sequencefmf 가져와 BERT 모델이 요구하는 입력 형태로 구성할 수 있음

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

token_type_ids는 input_ids의 어느 부분이 첫번째 문장이고 어느 부분이 두번째 문자인지 모델에 알려줌.

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

### 문장 리스트 토큰화

In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding = True,
    truncation = True,
)

위 방법은 작동은 잘하지만, input_ids, attention_mask, token_type_ids, 데이터가 담겨진 다차원 리스트가 키로 지정된 tokenized_dataset이라는 별도의 파이썬 딕셔너리를 반환하는 단접이 존재한다. 또한, 이 밥법은 토큰화 하는동안 전체 데이터셋을 저장할 충분한 공간의 RAM이 있는 경우에만 작동한다. 반면 datasets 라이브러리의 데이터셋들은 디스크에 저장된 파일이므로, 요청한 샘플만 메모리에 로드된 상태로 유지한다.  
특정 데이터를 dataset 객체로 유지하기 위해서는 Dataset.map() method를 사용하는데, 이 방법은 토큰화 외에도 더 많은 전처리가 필요한 경우 유연성을 발휘한다. map() method는 dataset의 개별 요소에 함수를 적용하여 작동하므로 입력을 토큰화하는 함수를 정의해보겠음.

In [ ]:
def tokenize_function(example):
  return tokenizer(example['sentence1'], example['sentence2'], truncation = True)

이 함수는 데이터셋의 개별 항목이 담겨진 딕셔너리를 매개변수로 입력 받아 input_ids, attention_mask 및 token_type_ids 키가 지정된 새로운 딕셔너리를 반환한다. 이처럼 토크나이저는 문장 쌍 리스트에서 작동하기 때문에 example dictionary가 여러 샘플인 경우에도 작동함.  
이렇게 하면 후술할 map()에서 batched = True 옵션을 사용할 수 있기에 토큰화 속도도 빨라짐.  
추가로 batch 형태로 실행을 하는 경우 전체 데이터셋의 최대 길이에 맞게끔 padding을 진행할 필요 없이 배치 내에서의 최대 길이에 맞게끔 padding을 진행하면 되기 대문에 시간을 매우 절약할 수 있음. 
다음은 한 번에 모든 데이터셋에 토큰화 기능을 적용하는 방법으로, map() 내에서 batched = True 옵션을 줘 함수가 각 요소에 개별적으로 적용되지 않고 batch 내 모든 요소들에 한꺼번에 적용됨. 이를 통해 더욱 빠른 전처리가 가능함.

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

## Dynamic Padding
: 전체 요소들을 배치로 분리했을 때, 가장 긴 요소의 길이로 모든 예제를 채우는 padding 방식

이를 위해서는 데이터셋의 요소 각가에 대해 정확한 수의 패딩을 적용할 수 있는 collate function을 정의해야 하는데, Transformers library의 DataCollatorWithPadding을 통해 이 기능을 사용할 수 있다. 

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

#2. Trainer API를 이용한 Fine Tuning


예제 실행을 위해 필요한 이전 내용들

In [ ]:
from datasets import load_dataset 
from transformers import AutoTokenizer, DataCollatorWithPadding 

raw_datasets = load_dataset("glue", "mrpc") 
checkpoint = "bert-base-uncased" 

tokenizer = AutoTokenizer.from_pretrained(checkpoint) 

def tokenize_function(example): 
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True) 
  
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) 
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Training

Trainer를 정의하기 이전에 Trainer가 학습/평가에 사용할 모든 hyperparameter를 포함하는 TrainingArguments 클래스를 정의해야함. 우리가 제공해야할 것은 학습된 모델이 저장될 디렉토리 뿐임.

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

모델 정의

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

지금까지 구성된 개체들을 이용하여 Trainer 정의

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model, 
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator, # default가 이전에 정의한 DataCollatorWithPadding이기 때문에 생략 가능
    tokenizer = tokenizer,
)

Fine Tuning

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377
  Number of trainable parameters = 109483778
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__cal

Step,Training Loss
500,0.615900
1000,0.497400


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1377, training_loss=0.4994832733531057, metrics={'train_runtime': 198.625, 'train_samples_per_second': 55.401, 'train_steps_per_second': 6.933, 'total_flos': 406183858377360.0, 'train_loss': 0.4994832733531057, 'epoch': 3.0})

# Evaluation

In [ ]:
predictions = trainer.predict(tokenized_datasets['validation'])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 408
  Batch size = 8


(408, 2) (408,)


predict method의 출력은 3개의 field(predictions, label_ids, metrics)가 있는 또 다른 named tuple이다.
* metrics: 전달된 dataset의 loss와 time metric(예측에 걸린 전체, 평균 시간)이 포함  
comput metrics()함수 구성 후 Trainer에 전달하면 해당 필드에는 compute_metrics()에서 반환한 metric도 포함됨
  

Transformer 모델은 logit을 반환하므로 이 logit값들을 label과 비교할 수 있는 예측 결과로 변환하기 위해 second axis에 존재하는 최대값이 있는 index를 가져와야 함

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis = -1)

compute_metric() 함수 구성을 위하여 load_metric()을 이용하여 MRPC dataset과 관련된 metric load. load된 metric에는 계산을 수행하기 위해 사용할 수 있는 compute() method가 있다.

In [ ]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions = preds, references = predictions.label_ids)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


{'accuracy': 0.8235294117647058, 'f1': 0.8795986622073578}

지금까지의 설명을 모두 종합하여 compute_metrics() 구성

In [ ]:
def compute_metrics(eval_preds):
  metric = load_metric("glue", "mrpc")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis = -1)
  return metric.compute(predictions = predictions, references = labels)

In [ ]:
training_args = TrainingArguments("test-traier", evaluation_strategy = "epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

trainer = Trainer(
    model, 
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"], 
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embeddi

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377
  Number of trainable parameters = 109483778


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.449976,0.789216,0.858553
2,0.577000,0.418979,0.825980,0.883031
3,0.383700,0.599368,0.848039,0.894198


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test-traier/checkpoint-500
Configuration saved in test-traier/checkpoint-500/config.json
Model weights saved in test-traier/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-traier/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-traier/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore t

TrainOutput(global_step=1377, training_loss=0.4133429212161572, metrics={'train_runtime': 206.4931, 'train_samples_per_second': 53.29, 'train_steps_per_second': 6.669, 'total_flos': 406183858377360.0, 'train_loss': 0.4133429212161572, 'epoch': 3.0})

#3. 전체 학습

Trainer 클래스를 사용하지 않고 이전 섹션에서 했던 것과 동일한 결과 얻기  
아래는 이번 섹션을 공부하기 위해 필요한 모든 작업을 수행하는 코드

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation = True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

실제 Training loop를 작성하기 전에 몇 가지 객체를 정의해야 함
1. Dataloaders  
그러나 Dataloaders를 정의하기 이전에 Trainer가 자동으로 수행해주었던 몇 가지 작업을 직접 처리하기 위해 tokenized_datasets에 약간의 후처리를 적용해주어야함. 
* 모델이 필요로 하지 않는 column 제거(sentence1, sentence2)
* 모델이 labels라는 이름으로 매개변수를 받기 때문에 column label의 이름을 labels로 바꾸어주기
* Python list가 아닌 Pytorch Tensor를 return하도록 datasets의 형식 설정
   
tokenized_datasets에는 이러한 작업을 수행하기 위한 별도의 method들이 존재한다.


## 학습을 위한 준비

In [2]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"]) # sentence1, sentence2, idx column 제거
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") # label column을 labels로 rename
tokenized_datasets.set_format("torch") # tensor가 return되도록 설정
tokenized_datasets["train"].column_names 

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

이제 dataloaders를 정의해보자

In [3]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle = True,
    batch_size = 8, 
    collate_fn = data_collator   
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size = 8,
    collate_fn = data_collator,
)

데이터 처리가 잘 되었는지 확인하기 위해 batch 검사

In [4]:
for batch in train_dataloader:
  break

{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 78]),
 'token_type_ids': torch.Size([8, 78]),
 'attention_mask': torch.Size([8, 78])}

모델 인스턴스화

In [5]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

학습 과정에서 모든 것들이 원활하게 진행될 수 있는지 확인하기 위해 batch를 전달해 결과 보기. 모델의 매개변수에 labels가 포함되어있다면 loss와 logit을 반환한다. 이 때 logit은 label이 2개이므로 8 x 2의 텐서로 반환된다.

In [6]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr = 5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


마지막으로, Trainer에서 default로 사용되는 learning rate schedulers는 최대값 5e-5에서 0까지 linear decay한다. 이를 적절하게 정의하기 위해서는 수행할 학습 단계의 횟수를 알아야하는데, 이는 # of epochs x # of batches이다. # of batches는 dataloader의 길이와 같다. Trainer는 default로 3개의 epoch을 사용하므로 다음을 따른다. 

In [7]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)
print(num_training_steps)

1377


## Training Loop

In [8]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [9]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

## Evaluation Loop

In [11]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim = -1)
  metric.add_batch(predictions = predictions, references = batch["labels"])

metric.compute()

{'accuracy': 0.8848039215686274, 'f1': 0.9188255613126081}

## Accelerate 라이브러리를 사용한 학습 루프 가속화

In [13]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)
optimizer = AdamW(model.parameters(), lr = 3e-5)

# 입력한 객체들을 적절한 컨테이너 감싸 분산 학습이 의도대로 작동되도록 함
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

## 총 정리

In [19]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler

import torch
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

# 데이터셋 적재
raw_datasets = load_dataset("glue", "mrpc")
# pretrained language model checkpoint이름 지정
checkpoint = "bert-base-uncased"
# 지정된 pretrained language model에서 tokenizer instantiate
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 토크나이저 함수 사용자 정의화(sentece1, sentence2 column에 대해서만 tokenizing)
def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation = True)

# 토크나이징
tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)
# batch별 padding을 위한 data collator 정의
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

# 불필요한 columns를 제거하고 pretrained language model만 남김
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# 데이터셋의 label column을 labels로 변경
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# 데이터셋의 유형을 PyTorch tensor로 변경
tokenized_datasets.set_format("torch")

# 변경된 column 출력
print(tokenized_datasets["train"].column_names)

# train dataloader 생성
train_dataloader = DataLoader(tokenized_datasets["train"],
                              shuffle = True, 
                              batch_size = 8, 
                              collate_fn = data_collator)

# evaluation dataloader 생성
eval_dataloader = DataLoader(tokenized_datasets["validation"],
                              shuffle = True, 
                              batch_size = 8, 
                              collate_fn = data_collator)

# Pretrained language model 인스턴스화
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)
# optimization function 정의
optimizer = AdamW(model.parameters(), lr = 5e-5)

# epoch 개수 설정
num_epochs = 3
# training step 수 계산
num_training_steps = num_epochs * len(train_dataloader)
# learning rate scheduler 설정
lr_scheduler = get_scheduler("linear", optimizer = optimizer, num_warmup_steps = 0, num_training_steps = num_training_steps)

# GPU로 모델 이동
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 진행 상황바 정의
progress_bar = tqdm(range(num_training_steps))

# 모델을 학습 모드로 전환
model.train()

# 학습 루프 시작
for epoch in range(num_epochs):
  for batch in train_dataloader:
    # 현재 배치 중에서 입력값을 모두 GPU로 이동
    batch = {k: v.to(device) for k, v in batch.items()}
    # 모델 실행
    outputs = model(**batch)
    # loss 가져오기
    loss = outputs.loss
    # backpropagation 수행
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

# 평가 메트릭 가져오기
metric = load_metric("glue", "mrpc")
# 모델을 평가 모드로 전환
model.eval()
for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim = -1)
  metric.add_batch(predictions = predictions, references = batch["labels"])

# 평가 결과 계산 및 출력
metric.compute()

  0%|          | 0/3 [00:00<?, ?it/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.8602941176470589, 'f1': 0.900523560209424}